In [187]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

Setting text properties for plots

In [188]:
plt.rcParams.update({'font.size': 26})
plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
plt.close()

Importing dataset

In [189]:
# Specifying simulation directory and the directory to save results in
wdir = str(input('Enter simulation directory path: '))

# Specifying a snapshot for temporal analysis
sdir = wdir + 'temporal_analysis/'

# Create a directory to store correlation plots
if not os.path.exists(sdir + 'correlation_plots/'):
    os.makedirs(sdir + 'correlation_plots/')

Enter simulation directory path: /Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Code+Results/Results/m11b_res2100_cr_heating_fix/


In [190]:
# Read in all metals and find store directories to all metals
metal_df = pd.read_csv((sdir + 'metal_list.csv'))
metals = metal_df['metals'].to_list()

spath_metals = {}

for m in metals:
    spath_metals[m] = sdir + m + '/'

In [191]:
# Obtained statistics for the rendered snapshots

rendered_df = pd.read_csv(sdir + 'rendered_snap_stats.csv')

print(rendered_df)

params_df = {}

for m in metals:
    params_df[m] = pd.read_csv(spath_metals[m] + 'data/fit/fit_{}_params.csv'.format(m))

     Unnamed: 0  snap     halo_mass      redshift       time  velocity_mass  \
0             0    80  5.736586e+09  4.338983e+00   1.439067      82.371468   
1             1    85  5.620534e+09  4.121951e+00   1.530909      80.267014   
2             2    90  6.047046e+09  3.923077e+00   1.623934      79.587395   
3             3    95  6.995570e+09  3.740741e+00   1.717761      77.904190   
4             4   100  8.517658e+09  3.571429e+00   1.813212      78.454208   
..          ...   ...           ...           ...        ...            ...   
100         100   580  4.568081e+10  1.782818e-02  13.553508     110.529854   
101         101   585  4.578685e+10  9.646261e-03  13.665295     111.209183   
102         102   590  4.588703e+10  1.594940e-03  13.776556     112.000145   
103         103   595  4.589751e+10  7.968344e-04  13.787654     112.089027   
104         104   600  4.590833e+10  4.440892e-16  13.798747     112.175308   

     velocity_vol  velocity_spread  sound_mass  sou

In [192]:
# Correlating abundance spread of all elements with each other

columns_spread = ['spread_' + m for m in metals]

spread_df = pd.DataFrame(columns = columns_spread)

for m in metals:
    spread_df['spread_' + m] = params_df[m]['std']
    
print(spread_df)
    
spread_corr = spread_df.corr()

plt.subplots(figsize = (13, 13))
sns.heatmap(spread_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/spread_corr.png')
plt.close()

     spread_c  spread_n  spread_o  spread_ne  spread_mg  spread_si  spread_s  \
0    0.273186  0.277966  0.287693   0.287852   0.286844   0.283566  0.280925   
1    0.329178  0.341778  0.329641   0.330077   0.328302   0.326794  0.325243   
2    0.294792  0.314113  0.305549   0.305785   0.304577   0.299004  0.295357   
3    0.432647  0.459161  0.374524   0.374270   0.370631   0.361504  0.354545   
4    0.417466  0.452947  0.327801   0.327134   0.324317   0.326744  0.322831   
..        ...       ...       ...        ...        ...        ...       ...   
100  0.242832  0.249506  0.213795   0.212568   0.213421   0.234847  0.240933   
101  0.262740  0.271493  0.225058   0.223508   0.224647   0.250936  0.258290   
102  0.248643  0.258109  0.207948   0.206245   0.207560   0.236158  0.244118   
103  0.243870  0.253172  0.203835   0.202164   0.203458   0.231617  0.239468   
104  0.241523  0.250847  0.201461   0.199791   0.201091   0.229205  0.237038   

     spread_ca  spread_fe  
0     0.277

In [193]:
columns_cent_mass = ['cent_mass_' + m for m in metals]

cent_mass_df = pd.DataFrame(columns = columns_cent_mass)

for m in metals:
    cent_mass_df['cent_mass_' + m] = params_df[m]['median_mass']
    
print(cent_mass_df)
    
cent_mass_corr = cent_mass_df.corr()

plt.subplots(figsize = (13, 13))
sns.heatmap(cent_mass_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/cent_mass_corr.png')
plt.close()

     cent_mass_c  cent_mass_n  cent_mass_o  cent_mass_ne  cent_mass_mg  \
0      -1.839405    -2.214950    -1.564205     -1.497055     -1.739327   
1      -1.966009    -2.332348    -1.739063     -1.673736     -1.914229   
2      -1.729282    -2.093389    -1.475868     -1.408216     -1.650392   
3      -2.270604    -2.486179    -2.457338     -2.388609     -2.624680   
4      -2.011971    -2.280942    -2.094209     -2.027024     -2.265610   
..           ...          ...          ...           ...           ...   
100    -1.424815    -1.672413    -1.408894     -1.345314     -1.583906   
101    -1.373744    -1.619641    -1.366954     -1.304099     -1.542237   
102    -1.263021    -1.506062    -1.270782     -1.208727     -1.446109   
103    -1.258691    -1.501461    -1.267928     -1.206247     -1.443540   
104    -1.257325    -1.500257    -1.267199     -1.205632     -1.442790   

     cent_mass_si  cent_mass_s  cent_mass_ca  cent_mass_fe  
0       -1.725181    -1.779511     -2.057963     -

In [194]:
columns_cent_vol = ['cent_vol_' + m for m in metals]

cent_vol_df = pd.DataFrame(columns = columns_cent_vol)

for m in metals:
    cent_vol_df['cent_vol_' + m] = params_df[m]['median_vol']
    
print(cent_vol_df)
    
cent_vol_corr = cent_vol_df.corr()

plt.subplots(figsize = (13, 13))
sns.heatmap(cent_vol_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/cent_vol_corr.png')
plt.close()

     cent_vol_c  cent_vol_n  cent_vol_o  cent_vol_ne  cent_vol_mg  \
0     -1.859261   -2.215861   -1.602841    -1.535623    -1.777765   
1     -1.964802   -2.312830   -1.803222    -1.737639    -1.978045   
2     -1.931568   -2.253857   -1.763793    -1.696268    -1.937717   
3     -2.205176   -2.473667   -2.458245    -2.388693    -2.624727   
4     -2.254813   -2.596703   -2.143177    -2.076087    -2.314845   
..          ...         ...         ...          ...          ...   
100   -1.301703   -1.545349   -1.304868    -1.242274    -1.480239   
101   -1.498332   -1.749532   -1.467097    -1.403029    -1.641912   
102   -1.392871   -1.638856   -1.383503    -1.320710    -1.558681   
103   -1.378187   -1.623371   -1.371075    -1.308080    -1.546246   
104   -1.369584   -1.614709   -1.364522    -1.301812    -1.539715   

     cent_vol_si  cent_vol_s  cent_vol_ca  cent_vol_fe  
0      -1.761637   -1.813801    -2.091517    -2.103392  
1      -1.931435   -1.965122    -2.230508    -2.194937  


In [195]:
cent_mass_cent_vol_corr = pd.concat([cent_vol_df, cent_mass_df], axis=1, 
                                keys=['cent_vol_df', 'cent_mass_df']).corr().loc['cent_vol_df', 'cent_mass_df']
print(cent_mass_cent_vol_corr)

plt.subplots(figsize = (13, 13))
sns.heatmap(cent_mass_cent_vol_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/cent_mass_cent_vol_corr.png')
plt.close()

             cent_mass_c  cent_mass_n  cent_mass_o  cent_mass_ne  \
cent_vol_c      0.911602     0.914912     0.825118      0.823178   
cent_vol_n      0.920223     0.929146     0.812194      0.809911   
cent_vol_o      0.858979     0.835051     0.884924      0.884755   
cent_vol_ne     0.857248     0.833095     0.884528      0.884422   
cent_vol_mg     0.859100     0.835352     0.884429      0.884269   
cent_vol_si     0.884259     0.866571     0.879297      0.878106   
cent_vol_s      0.889783     0.874081     0.875503      0.874004   
cent_vol_ca     0.893882     0.880122     0.870613      0.868832   
cent_vol_fe     0.909638     0.906467     0.839963      0.836791   

             cent_mass_mg  cent_mass_si  cent_mass_s  cent_mass_ca  \
cent_vol_c       0.825730      0.858023     0.865253      0.870556   
cent_vol_n       0.812861      0.851542     0.860805      0.867998   
cent_vol_o       0.885154      0.883277     0.879609      0.874779   
cent_vol_ne      0.884771      0.881964

In [196]:
spread_cent_mass_corr = pd.concat([cent_mass_df, spread_df], axis=1, keys=['cent_mass_df', 'spread_df']).corr().loc['cent_mass_df', 'spread_df']
print(spread_cent_mass_corr)

plt.subplots(figsize = (13, 13))
sns.heatmap(spread_cent_mass_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/spread_cent_mass_corr.png')
plt.close()

              spread_c  spread_n  spread_o  spread_ne  spread_mg  spread_si  \
cent_mass_c  -0.688352 -0.722107 -0.705659  -0.708627  -0.706182  -0.643572   
cent_mass_n  -0.674698 -0.705816 -0.706360  -0.709543  -0.707105  -0.641784   
cent_mass_o  -0.692363 -0.731397 -0.658790  -0.660771  -0.658319  -0.610502   
cent_mass_ne -0.693524 -0.732276 -0.659660  -0.661596  -0.659200  -0.612189   
cent_mass_mg -0.693187 -0.732088 -0.659983  -0.661961  -0.659527  -0.611814   
cent_mass_si -0.681413 -0.723420 -0.654464  -0.656968  -0.653957  -0.596636   
cent_mass_s  -0.675778 -0.718510 -0.651334  -0.654009  -0.650831  -0.590644   
cent_mass_ca -0.670375 -0.713757 -0.647738  -0.650532  -0.647233  -0.584934   
cent_mass_fe -0.629770 -0.674621 -0.622566  -0.626151  -0.622155  -0.547417   

              spread_s  spread_ca  spread_fe  
cent_mass_c  -0.623057  -0.602343  -0.509085  
cent_mass_n  -0.620431  -0.598979  -0.498348  
cent_mass_o  -0.594488  -0.577832  -0.518983  
cent_mass_ne -0.59645

In [197]:
spread_cent_vol_corr = pd.concat([cent_vol_df, spread_df], axis=1, keys=['cent_vol_df', 'spread_df']).corr().loc['cent_vol_df', 'spread_df']
print(spread_cent_vol_corr)

plt.subplots(figsize = (13, 13))
sns.heatmap(spread_cent_vol_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/spread_cent_vol_corr.png')
plt.close()

             spread_c  spread_n  spread_o  spread_ne  spread_mg  spread_si  \
cent_vol_c  -0.801250 -0.818591 -0.805316  -0.807460  -0.804724  -0.756493   
cent_vol_n  -0.780249 -0.799638 -0.791675  -0.794021  -0.791316  -0.739767   
cent_vol_o  -0.837128 -0.849722 -0.814739  -0.816201  -0.813236  -0.775689   
cent_vol_ne -0.838380 -0.850598 -0.816038  -0.817468  -0.814537  -0.777552   
cent_vol_mg -0.837437 -0.849924 -0.815391  -0.816851  -0.813900  -0.776424   
cent_vol_si -0.815841 -0.833500 -0.795362  -0.797204  -0.793898  -0.749097   
cent_vol_s  -0.806977 -0.826160 -0.787557  -0.789541  -0.786116  -0.738821   
cent_vol_ca -0.798756 -0.819336 -0.779927  -0.782011  -0.778504  -0.729353   
cent_vol_fe -0.743024 -0.770420 -0.733292  -0.736154  -0.732105  -0.670300   

             spread_s  spread_ca  spread_fe  
cent_vol_c  -0.737281  -0.716698  -0.631913  
cent_vol_n  -0.719867  -0.698782  -0.608839  
cent_vol_o  -0.759157  -0.740724  -0.678334  
cent_vol_ne -0.761198  -0.742934  -

In [198]:
fe_columns = ['spread_fe', 'cent_mass_fe', 'cent_vol_fe']
fe_df = pd.DataFrame({'spread_fe': params_df['fe']['std'], 'cent_mass_fe': params_df['fe']['median_mass'],
                     'cent_vol_fe': params_df['fe']['median_vol']})

print(fe_df)

     spread_fe  cent_mass_fe  cent_vol_fe
0     0.274597     -2.077824    -2.103392
1     0.346579     -2.171545    -2.194937
2     0.279807     -2.015325    -2.237361
3     0.360654     -2.811920    -2.799040
4     0.342729     -2.498913    -2.582295
..         ...           ...          ...
100   0.284230     -1.589381    -1.440599
101   0.308524     -1.526560    -1.681376
102   0.295997     -1.391815    -1.548023
103   0.290500     -1.386817    -1.530677
104   0.287762     -1.385315    -1.522181

[105 rows x 3 columns]


In [199]:
SFR_df = rendered_df[['SFR@10Myr', 'SFR@100Myr', 'SFR@1000Myr']].copy()

SFR_chem_corr = pd.concat([fe_df, SFR_df], axis=1, keys=['fe_df', 'SFR_df']).corr().loc['fe_df', 'SFR_df']
print(SFR_chem_corr)

plt.subplots(figsize = (13, 13))
sns.heatmap(SFR_chem_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/SFR_fe_corr.png')
plt.close()

              SFR@10Myr  SFR@100Myr  SFR@1000Myr
spread_fe      0.274170    0.140103     0.342408
cent_mass_fe  -0.414053   -0.511329    -0.872767
cent_vol_fe   -0.485037   -0.490782    -0.841936


In [200]:
halo_df = rendered_df[['halo_mass']].copy()

halo_chem_corr = pd.concat([fe_df, halo_df], axis=1, keys=['fe_df', 'halo_df']).corr().loc['fe_df', 'halo_df']
print(halo_chem_corr)

plt.subplots(figsize = (13, 13))
sns.heatmap(halo_chem_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/halo_fe_corr.png')
plt.close()

              halo_mass
spread_fe     -0.197443
cent_mass_fe   0.886008
cent_vol_fe    0.839713


In [201]:
velo_df = rendered_df[['velocity_mass', 'sound_mass', 'thermal_mass',
                      'velocity_vol', 'sound_vol', 'thermal_vol']].copy()

velo_chem_corr = pd.concat([fe_df, velo_df], axis=1, keys=['fe_df', 'velo_df']).corr().loc['fe_df', 'velo_df']
print(velo_chem_corr)

plt.subplots(figsize = (13, 13))
sns.heatmap(velo_chem_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/velo_fe_corr.png')
plt.close()

              velocity_mass  sound_mass  thermal_mass  velocity_vol  \
spread_fe          0.351429   -0.042216     -0.042215      0.360447   
cent_mass_fe       0.523551   -0.763940     -0.763939      0.547088   
cent_vol_fe        0.448705   -0.703131     -0.703131      0.467963   

              sound_vol  thermal_vol  
spread_fe      0.110605     0.110605  
cent_mass_fe   0.710598     0.710598  
cent_vol_fe    0.649466     0.649466  


In [202]:
mach_df = rendered_df[['mach_number_mass']].copy()

mach_chem_corr = pd.concat([fe_df, mach_df], axis=1, keys=['fe_df', 'mach_df']).corr().loc['fe_df', 'mach_df']
print(mach_chem_corr)

plt.subplots(figsize = (13, 13))
sns.heatmap(mach_chem_corr, annot = True, square = True)
plt.savefig(sdir + 'correlation_plots/mach_fe_corr.png')
plt.close()

              mach_number_mass
spread_fe             0.208558
cent_mass_fe          0.668811
cent_vol_fe           0.598833
